# TechBairn Project-3

Made by: Rishabhraj Srivastava

### Importing Libraries

In [3]:
# Import the necessary libraries
import torch
import torchvision
import torchvision.transforms as transforms

### Defining the Hyper Parameters

In [9]:
batch_size = 32 # The number of images in each batch
num_epochs = 10 # The number of times to train on the whole dataset
learning_rate = 0.01 # The learning rate for the optimizer
num_classes = 2 # The number of classes (apple or banana)

In [11]:
# Define the transformations for the images
transform = transforms.Compose(
    [transforms.Resize(224), # Resize the images to 224 x 224 pixels
     transforms.ToTensor(), # Convert the images to tensors
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Normalize the images

# Load the training and testing datasets from the folders
trainset = torchvision.datasets.ImageFolder(root='/Users/rishabhrajsrivastava/Fruit-Images-Dataset/Training', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/Users/rishabhrajsrivastava/Fruit-Images-Dataset/Test', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2

# Define the classes (Apple and Banana)
classes = ('Apple', 'Banana')

### Creating a Model

In [12]:
# Define the CNN model using PyTorch
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Convolutional layers
        self.conv1 = torch.nn.Conv2d(3, 16, 3) # Input channels: 3, Output channels: 16, Kernel size: 3
        self.conv2 = torch.nn.Conv2d(16, 32, 3) # Input channels: 16, Output channels: 32, Kernel size: 3
        self.conv3 = torch.nn.Conv2d(32, 64, 3) # Input channels: 32, Output channels: 64, Kernel size: 3
        # Pooling layer
        self.pool = torch.nn.MaxPool2d(2, 2) # Kernel size: 2, Stride: 2
        # Fully connected layers
        self.fc1 = torch.nn.Linear(64 * 26 * 26, 256) # Input features: 64 * 26 * 26, Output features: 256
        self.fc2 = torch.nn.Linear(256, num_classes) # Input features: 256, Output features: num_classes
        # Dropout layer
        self.dropout = torch.nn.Dropout(0.2) # Dropout probability: 0.2
        
    def forward(self, x):
        # Apply the convolutional layers with ReLU activation and pooling
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = self.pool(torch.nn.functional.relu(self.conv3(x)))
        # Flatten the output of the last convolutional layer
        x = x.view(-1, 64 * 26 * 26)
        # Apply the dropout layer
        x = self.dropout(x)
        # Apply the first fully connected layer with ReLU activation
        x = torch.nn.functional.relu(self.fc1(x))
        # Apply the dropout layer
        x = self.dropout(x)
        # Apply the second fully connected layer with softmax activation
        x = torch.nn.functional.softmax(self.fc2(x), dim=1)
        return x

### Creating an Instance, Training and Saving the Model

In [13]:
# Create an instance of the CNN model
model = CNN()

# Define the loss function (cross entropy loss)
criterion = torch.nn.CrossEntropyLoss()

# Define the optimizer (stochastic gradient descent)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model on the training dataset
for epoch in range(num_epochs):
    running_loss = 0.0 # The running loss for each epoch
    for i, data in enumerate(trainloader):
        inputs, labels = data # Get the inputs and labels from the data
        optimizer.zero_grad() # Zero the parameter gradients
        outputs = model(inputs) # Forward pass the inputs through the model
        loss = criterion(outputs, labels) # Calculate the loss
        loss.backward() # Backward pass the loss through the model
        optimizer.step() # Update the parameters with the optimizer

        running_loss += loss.item() # Add the loss to the running loss

        if i % 200 == 199: # Print the statistics every 200 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

# Save the model
torch.save(model, 'model.pth')
print('Saved Model')

Finished Training
Saved Model


### Testing Model Based upon the Training Dataset

In [17]:
# Test the model on the testing dataset
correct = 0 # The number of correct predictions
total = 0 # The total number of predictions
with torch.no_grad(): # No need to calculate the gradients
    for data in testloader:
        images, labels = data # Get the images and labels from the data
        outputs = model(images) # Forward pass the images through the model
        _, predicted = torch.max(outputs.data, 1) # Get the predicted class with the maximum probability
        total += labels.size(0) # Add the number of labels to the total
        correct += (predicted == labels).sum().item() # Add the number of correct predictions to the correct

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))


Accuracy of the network on the test images: 100 %


### Training Accuracy of each Class

In [18]:
# Print the accuracy for each class
class_correct = list(0. for i in range(num_classes)) # The number of correct predictions for each class
class_total = list(0. for i in range(num_classes)) # The total number of predictions for each class
with torch.no_grad(): # No need to calculate the gradients
    for data in testloader:
        images, labels = data # Get the images and labels from the data
        outputs = model(images) # Forward pass the images through the model
        _, predicted = torch.max(outputs, 1) # Get the predicted class with the maximum probability
        c = (predicted == labels).squeeze() # Get a tensor of 0s and 1s indicating the correctness of each prediction
        for i in range(min(batch_size, len(labels))): # Loop over each image in the batch
            label = labels[i] # Get the label of the image
            class_correct[label] += c[i].item() # Add the correctness to the class_correct list
            class_total[label] += 1 # Add one to the class_total list

for i in range(num_classes): # Loop over each class
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of Apple : 100 %
Accuracy of Banana : 100 %


### Classifying the Image

In [ ]:
# Import the necessary libraries
import PIL # For loading and processing images
import numpy as np # For converting images to arrays

# Load the model
model = torch.load('model.pth')
print('Loaded Model')

# Define a function to take an image as input and classify it
def classify_image(image_path):
    # Load the image from the path
    image = PIL.Image.open(image_path)
    # Apply the same transformations as the training and testing datasets
    image = transform(image)
    # Add a batch dimension to the image tensor
    image = image.unsqueeze(0)
    # Forward pass the image through the model
    output = model(image)
    # Get the predicted class with the maximum probability
    _, predicted = torch.max(output.data, 1)
    # Get the class name from the classes list
    class_name = classes[predicted.item()]
    # Print the result
    print('The image is classified as:', class_name)

# Test the function with an example image
classify_image('example.jpg')